**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [10]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 22:15:43,106 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 22:15:43.107 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [13]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-09 22:15:43.204 | INFO     | __main__:<module>:13 - User Role: author
2025-02-09 22:15:43.207 | INFO     | __main__:<module>:24 - Creating account with name: exciting_hofstadter
2025-02-09 22:15:43.209 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 22:15:43.227 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 22:15:43.229 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:15:43.230 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:15:43.231 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-09 22:15:43,248 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:15:43,249 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 

2025-02-09 22:15:46,198 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:15:46,206 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:15:46.210 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 2
2025-02-09 22:15:46.211 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


3 - Queries the blockchain to confirm the succesful creation of the account.

In [14]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-09 22:15:46.230 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:15:46,233 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:15:46,253 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:15:46,255 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:15:49,232 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:15:49,241 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:15:49,243 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '3df30973a4be83f31c3fa8e105141433b7267d0a6c22bf9e89b580b85b110f7c', 'timestamp': 1739139349243}
2025-02-09 22:15:49.245 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:15:49,249 - INFO - Appended data under 'exciting_hofstadter@test' in logs/account_data.json
2025-02-09 22:15:49.250 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-0

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 22:15:49,301 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 22:15:49.302 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [16]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-09 22:15:49.356 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 22:15:49.381 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 22:15:49.383 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 22:15:49.385 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:15:49.387 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-09 22:15:49,390 - INFO - Processing account: exciting_hofstadter@test
2025-02-09 22:15:49,392 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Exciting Hofstadter', 'foaf:mbox': 'exciting_hofstadter@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Dhurakijpundit Universi

2025-02-09 22:15:52,263 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:15:52,272 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:15:52.274 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 22:15:52.295 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 22:15:52.296 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-09 22:15:52.297 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-09 22:15:52.298 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-09 22:15:52.300 | DEBUG    | integration_helpers

6 - Queries the blockchain again to confirm the succesfull details setting.

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-02-09 22:15:52.317 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:15:52,320 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:15:52,339 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:15:52,342 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:15:55,294 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:15:55,302 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:15:55,304 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'fc39510904898a8e6fb892506921a5000717375c9bf26777e7080bdee86d674f', 'timestamp': 1739139355304}
2025-02-09 22:15:55.305 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:15:55,308 - INFO - Appended data under 'exciting_hofstadter@test' in logs/account_data.json
2025-02-09 22:15:55.310 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-0

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [18]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-09 22:15:55.366 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'exciting_hofstadter@test', 'schema:publicKey': 'dbe5043db833486921eb5604149023d41619bbe4c9156240b6c7d5832b9a942b', 'schema:roleName': 'author'}, 'foaf:mbox': 'exciting_hofstadter@email.com', 'foaf:name': 'Exciting Hofstadter', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Dhurakijpundit University'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '0505-8709-9666-X'}}
